# Power calculations for a clinical trial

Here we're going to:
- run a power calculation for a phase 3 clinical trial (2 independent groups)
- simulate the trial given our assumptions (using Monte Carlo) and validate our design

If we get our power calculations right, then we should be able to predict the chance of finding a true difference between groups (**power**) and the chance of falsely finding a difference between groups when it doesn't actually exist (**alpha**)

## Step 1: Import some useful libraries

In [ ]:
import scipy.stats as stats
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rcParams
from IPython.display import HTML
import warnings
warnings.filterwarnings('ignore')

## Step 2: Power calcuation

Here we will decide what the mean outcome measure will be in **Group 1** and **Group 2**, as well as the variance of the outcome measure (expressed as a standard deviation, and here we assume it is equal for each group). 

In [ ]:
mean_group1 = 100 
mean_group2 = 110
sd_group = 10

Now we will decide how to set up our trial:
- Alpha (a.k.a. the significance level, or how often we detect a difference when one **doesn't** exist)
- Power (or how often we detect a difference when it **does** exist)
- Number of tails (are we looking for a one-sided or two-sided effect?)

In [ ]:
alpha = 0.05
power = 0.8
no_tails = 2

Now we can work out how many subjects we need for each group!

In the code below, we will keep increasing the number of subjects in each group until we reach the desired power (specified above)

In [ ]:
alpha_tail = alpha/no_tails

max_allowed = 2000
calc_power=np.zeros((max_allowed,1))
delta=np.zeros((max_allowed,1))
df=np.zeros((max_allowed,1))
tcrit=np.zeros((max_allowed,1))

cohens_d = np.abs(mean_group1-mean_group2)/sd_group

for n in np.arange(2,max_allowed+1,1):
    n1 = n
    n2 = n
    
    delta[n] = cohens_d*np.sqrt(1/((1/n1)+(1/n2)))
    df[n] = n1 + n2 - 2
    tcrit[n] = stats.t.ppf(1-alpha_tail,df[n])
    calc_power[n]=1-(stats.nct.cdf(tcrit[n],df[n],delta[n])-stats.nct.cdf(-tcrit[n],df[n],delta[n]))

    if calc_power[n] > power:
        print("We need at least "+str(n)+" per group")
        print(str(no_tails)+" tail alpha = "+str(alpha))
        print("Exact power = "+'{:.3f}'.format(calc_power[n][0]))
        break

We've done the power calculation! We now know how many subjects we need in each group to achieve the target alpha and power. We can also visualise what happened to the power as we kept changing the number of subjects in each group:

*(aside: we're only visualising up to 100 subjs per group so it doesn't take ages to plot...)*

In [ ]:
def power_calc_plot(t):
    zpts=np.arange(-10,10,0.01)
    ax1.clear()
    ax1.plot(zpts,stats.t.pdf(zpts,df[t]),'r')
    ax1.plot(zpts,stats.nct.pdf(zpts,df[t],delta[t]),'b')
    ax1.stem(tcrit[t],stats.nct.pdf(tcrit[t],df[t],delta[t]),'k',markerfmt=" ")
    ax1.text(tcrit[t]+1,0.3,"Power = "+'{:.2f}'.format(calc_power[t][0]))
    ax1.text(tcrit[t]+1,0.2,str(no_tails)+" tail alpha = "+str(alpha))
    ax1.set_ylabel('Prob. density')
    ax1.set_xlabel('z')
    #ax1.set_ylim([0,0.4])
    ax1.fill_between(zpts[zpts>tcrit[t]], stats.nct.pdf(zpts[zpts>tcrit[t]],df[t],delta[t]), 0, color='b', alpha=0.3)
    ax1.fill_between(zpts[zpts>tcrit[t]], stats.t.pdf(zpts[zpts>tcrit[t]],df[t]), 0, color='r', alpha=0.3)

    ax2.clear()
    ax2.plot(np.arange(2,n+1,1),calc_power[2:n+1],'k:')
    ax2.scatter(t,calc_power[t])
    ax2.text(t,calc_power[t]+0.1,str(t)+" per group")
    ax2.set_ylabel('Power')
    ax2.set_xlabel('Subjs per group')
    ax2.set_ylim([0,1])    
    ax2.set_xlim([2,n])    
    
if n<100:    
    fig, (ax1, ax2) = plt.subplots(2, 1)
    fig.subplots_adjust(hspace=0.25)
    html=HTML(animation.FuncAnimation(fig, power_calc_plot,frames=np.arange(2,n+1,1),init_func=lambda: None, interval=60).to_jshtml())
    display(html)
    plt.close() 
else:
    print('Too many points to visualise!')

## Step 3: Monte Carlo simulation (treatment effect)

Now we will run a Monte Carlo simulation: essentially we pretend to run the trial many, many times, using the outcome measure distributions we assumed in the power calculation, and see how often we measure a significant difference. This proportion should be close to the **power** we used in our study design.

First, we assume that the mean and standard deviations of each group are exactly as we predicted:

In [ ]:
# Assume there is a real difference between groups
sim_mean_group1 = mean_group1 
sim_mean_group2 = mean_group2
sim_sd_group = sd_group
sim_n1 = n
sim_n2 = n 

Now let's repeatedly run a trial, where the outcome measure data looks exactly as we predicted (i.e. has the same mean values and standard deviations)

In [ ]:
# Let's simulate our study lots of times!
trials_to_sim = 10000
trial_result = np.zeros((trials_to_sim,2))

# TESTING FOR A GROUP DIFFERENCE: WHAT IS OUR TRUE POWER?
for trial_no in range(trials_to_sim):
    
    data_gp1 = sim_mean_group1 + sim_sd_group*(np.random.normal(0., 1., sim_n1))
    data_gp2 = sim_mean_group2 + sim_sd_group*(np.random.normal(0., 1., sim_n2))
                                       
    trial_result[trial_no] = stats.ttest_ind(data_gp1, data_gp2)

meas_power = np.sum(trial_result[:,1]<0.05)/trials_to_sim
print("We measure a (true) effect in "+'{:.2f}'.format(meas_power*100)+"% of the simulated trials!")
print("We designed our study to have a power of "+'{:.2f}'.format(power*100)+"%")

## Step 4: Monte Carlo simulation (no treatment effect)

Let's repeat the Monte Carlo simulation, but this time assuming that both groups have the same mean outcome measure. This time there is *no treatment effect*, and so the proportion of times we (falsely) measure a significant effect should be close to the **alpha** we used in our study design.

In [ ]:
# Let's simulate our study lots of times!
trials_to_sim = 10000

# Assume there is a NO real difference between groups
sim_mean_group1 = mean_group1 
sim_mean_group2 = mean_group1 # NB: the groups now have the same mean!
sim_sd_group = sd_group
sim_n1 = n
sim_n2 = n 


trial_result = np.zeros((trials_to_sim,2))

# TESTING FOR A GROUP DIFFERENCE: WHAT IS OUR TRUE ALPHA?
for trial_no in range(trials_to_sim):
    
    data_gp1 = sim_mean_group1 + sim_sd_group*(np.random.normal(0., 1., sim_n1))
    data_gp2 = sim_mean_group2 + sim_sd_group*(np.random.normal(0., 1., sim_n2))
                                       
    trial_result[trial_no] = stats.ttest_ind(data_gp1, data_gp2)
                                       
meas_alpha = np.sum(trial_result[:,1]<0.05)/trials_to_sim

print("We measure a (non-existant) effect in "+'{:.2f}'.format(meas_alpha*100)+"% of trials!")
print("We designed our study to have an alpha of "+'{:.2f}'.format(alpha*100)+"%")